In [2]:
# data from http://grouplens.org/datasets/movielens/

In [3]:
import os
import numpy as np
#data_folder = os.path.join(os.path.expanduser("ml-100k\\")
data_folder = os.path.join(os.path.expanduser("~"), "Downloads\\fitchburg\\courses\\datamining\\")
print data_folder
ratings_filename = os.path.join(data_folder, "BX-Book-Ratings.csv")

C:\Users\swaroop\Downloads\fitchburg\courses\datamining\


In [4]:
import pandas as pd

In [5]:
#all_ratings = pd.read_csv(ratings_filename, delimiter=";", header=None, names = ["UserID", "MovieID", "Rating", "Datetime"])
all_ratings = pd.read_csv(ratings_filename, delimiter=";")
#all_ratings["Datetime"] = pd.to_datetime(all_ratings['Datetime'],unit='s')

all_ratings = all_ratings[np.isfinite(all_ratings['Book-Rating'])]
all_ratings['Book-Rating'] = all_ratings['Book-Rating'].apply(np.int64)

all_ratings["Favorable"] = all_ratings["Book-Rating"] > 3

all_ratings[:5]

,User-ID,ISBN,Book-Rating,Favorable
0,276725,034545104X,0,False
1,276726,0155061224,5,True
2,276727,0446520802,0,False
3,276729,052165615X,3,False
4,276729,0521795028,6,True


In [6]:
users_filename = os.path.join(data_folder, "BX-Users.csv")
users = pd.read_csv(users_filename, delimiter=";")

print users.shape
#users = users.dropna(subset= ["Location"]) #df.dropna()
users = users.dropna(subset= ["Age"])
users = users[np.isfinite(users['Age'])]
#users = users.dropna(subset= ["Age"])
print users.shape

result = pd.merge(all_ratings, users, how='inner', on=['User-ID'])




print all_ratings.shape
print result.shape

#0-5, 6-9, 10-12, 13-17, 18-25, 26-35, 36-55,55-70,70+
def label_race (row):
   if 0 <= row['Age'] <= 5 :
      return '0-5'
   elif 6 <= row['Age'] <= 9 :
      return '6-9'
   elif 10 <= row['Age'] <= 12 :
      return '10-12'
   elif 13 <= row['Age'] <= 17 :
      return '13-17'
   elif 18 <= row['Age'] <= 25 :
      return '18-25'
   elif 26 <= row['Age'] <= 35 :
      return '26-35'
   elif 36 <= row['Age'] <= 55 :
      return '36-55'
   elif 55 <= row['Age'] <= 70 :
      return '55-70'
   elif row['Age'] > 70 :
      return '70+'

result['Age_Group'] = result.apply (lambda row: label_race (row),axis=1)

print result.columns

result = result.dropna(subset= ["Age_Group"])

print result.shape

print result[:5] 


# Sample the dataset. You can try increasing the size of the sample, but the run time will be considerably longer
#ratings = all_ratings[all_ratings['User-ID'].isin(range(20000))]  # & 
ratings = result[result["User-ID"].isin(range(50000))]
#ratings = result.sample(frac=0.1, replace=False)
#ratings = result
print len(ratings)

# We start by creating a dataset of each user's favourable reviews
favorable_ratings = ratings[ratings["Favorable"]]
favorable_ratings[:5]

print len(favorable_ratings)

# We are only interested in the reviewers who have more than one review
favorable_reviews_by_users = dict((k, frozenset(v.values)) for k, v in favorable_ratings.groupby("Age_Group")["ISBN"])
len(favorable_reviews_by_users)


# Find out how many movies have favourable ratings
num_favorable_by_movie = ratings[["ISBN", "Favorable"]].groupby("ISBN").sum()
num_favorable_by_movie.sort("Favorable", ascending=False)[:5]

print len(num_favorable_by_movie)

print num_favorable_by_movie[:5]

from collections import defaultdict

def find_frequent_itemsets(favorable_reviews_by_users, k_1_itemsets, min_support):
    counts = defaultdict(int)
    for user, reviews in favorable_reviews_by_users.items():
        for itemset in k_1_itemsets:
            if itemset.issubset(reviews):
                for other_reviewed_movie in reviews - itemset:
                    current_superset = itemset | frozenset((other_reviewed_movie,))
                    counts[current_superset] += 1
    return dict([(itemset, frequency) for itemset, frequency in counts.items() if frequency >= min_support])


import sys
frequent_itemsets = {}  # itemsets are sorted by length
min_support = 12

# k=1 candidates are the isbns with more than min_support favourable reviews
frequent_itemsets[1] = dict((frozenset((ISBN,)), row["Favorable"])
                                for ISBN, row in num_favorable_by_movie.iterrows()
                                if row["Favorable"] > min_support)

print("There are {} books with more than {} favorable reviews".format(len(frequent_itemsets[1]), min_support))
sys.stdout.flush()
for k in range(2, 20):
    # Generate candidates of length k, using the frequent itemsets of length k-1
    # Only store the frequent itemsets
    cur_frequent_itemsets = find_frequent_itemsets(favorable_reviews_by_users, frequent_itemsets[k-1],
                                                   min_support)
    if len(cur_frequent_itemsets) == 0:
        print("Did not find any frequent itemsets of length {}".format(k))
        sys.stdout.flush()
        break
    else:
        print("I found {} frequent itemsets of length {}".format(len(cur_frequent_itemsets), k))
        #print(cur_frequent_itemsets)
        sys.stdout.flush()
        frequent_itemsets[k] = cur_frequent_itemsets
# We aren't interested in the itemsets of length 1, so remove those
del frequent_itemsets[1]

(278858, 3)
(168096, 3)
(1149780, 4)
(840288, 6)
Index([u'User-ID', u'ISBN', u'Book-Rating', u'Favorable', u'Location', u'Age',
       u'Age_Group'],
      dtype='object')
(840288, 7)
   User-ID        ISBN  Book-Rating Favorable  \
0   276727  0446520802            0     False   
1   276729  052165615X            3     False   
2   276729  0521795028            6      True   
3   276733  2080674722            0     False   
4   276737  0600570967            6      True   

                             Location   Age Age_Group  
0       h, new south wales, australia  16.0     13-17  
1                rijeka, n/a, croatia  16.0     13-17  
2                rijeka, n/a, croatia  16.0     13-17  
3                  paris, n/a, france  37.0     36-55  
4  sydney, new south wales, australia  14.0     13-17  
134966
50038
79562
               Favorable
ISBN                    
0 14 02.9640         1.0
0 7322 6794 3        0.0
0 75280 122 8        0.0
0 907 062 008        1.0
0/330/25864/8   

C:\Users\swaroop\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:71: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


There are 144 books with more than 12 favorable reviews
I found 8 frequent itemsets of length 2
I found 5 frequent itemsets of length 3
I found 1 frequent itemsets of length 4
Did not find any frequent itemsets of length 5


In [7]:
print("Found a total of {0} frequent itemsets".format(sum(len(itemsets) for itemsets in frequent_itemsets.values())))

# Now we create the association rules. First, they are candidates until the confidence has been tested
candidate_rules = []
for itemset_length, itemset_counts in frequent_itemsets.items():
    for itemset in itemset_counts.keys():
        for conclusion in itemset:
            premise = itemset - set((conclusion,))
            candidate_rules.append((premise, conclusion))
print("There are {} candidate rules".format(len(candidate_rules)))

# Now, we compute the confidence of each of these rules. This is very similar to what we did in chapter 1
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in favorable_reviews_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1
rule_confidence = {candidate_rule: correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
              for candidate_rule in candidate_rules}

			  
			  
# Choose only rules above a minimum confidence level
min_confidence = 0.9

# Filter out the rules with poor confidence
rule_confidence = {rule: confidence for rule, confidence in rule_confidence.items() if confidence > min_confidence}
print(len(rule_confidence))

from operator import itemgetter
sorted_confidence = sorted(rule_confidence.items(), key=itemgetter(1), reverse=True)

Found a total of 14 frequent itemsets
There are 35 candidate rules
26


In [10]:
movie_name_filename = os.path.join(data_folder, "BX-Books.csv")
movie_name_data = pd.read_csv(movie_name_filename, delimiter=";", encoding = "latin-1", error_bad_lines = False, warn_bad_lines = False)
#movie_name_data.columns = ["MovieID", "Title", "Release Date", "Video Release", "IMDB", "<UNK>", "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
print movie_name_data.shape
print movie_name_data.columns

def get_movie_name(movie_id):
    title_object = movie_name_data[movie_name_data["ISBN"] == str(movie_id)]["Book-Title"]
    title = title_object.values[0]
    return title

count = 0
while(count <5):  
  count += 1
  
  try:  
    print("Rule #{0}".format(count))
    (premise, conclusion) = sorted_confidence[count][0]
    print premise
    print conclusion
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(str(conclusion))
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Confidence: {0:.3f}".format(rule_confidence[(premise, conclusion)]))
    print("")
  except:
    random_flag = 1


(271360, 8)
Index([u'ISBN', u'Book-Title', u'Book-Author', u'Year-Of-Publication',
       u'Publisher', u'Image-URL-S', u'Image-URL-M', u'Image-URL-L'],
      dtype='object')
Rule #1
frozenset(['0446606812', '0316666343'])
0345417623
Rule: If a person recommends Message in a Bottle, The Lovely Bones: A Novel they will also recommend Timeline
 - Confidence: 1.000

Rule #2
frozenset(['0971880107', '0345417623'])
0316666343
Rule: If a person recommends Wild Animus, Timeline they will also recommend The Lovely Bones: A Novel
 - Confidence: 1.000

Rule #3
frozenset(['0446606812'])
0316666343
Rule: If a person recommends Message in a Bottle they will also recommend The Lovely Bones: A Novel
 - Confidence: 1.000

Rule #4
frozenset(['0446672211', '0345417623'])
0316666343
Rule: If a person recommends Where the Heart Is (Oprah's Book Club (Paperback)), Timeline they will also recommend The Lovely Bones: A Novel
 - Confidence: 1.000

Rule #5
frozenset(['0971880107', '0446672211'])
0345417623
Rul

In [11]:
# Evaluation using test data
test_dataset = all_ratings[~all_ratings['User-ID'].isin(range(50000))]
test_favorable = test_dataset[test_dataset["Favorable"]]
#test_not_favourable = test_dataset[~test_dataset["Favourable"]]
test_favorable_by_users = dict((k, frozenset(v.values)) for k, v in test_favorable.groupby("User-ID")["Book-Rating"])
#test_not_favourable_by_users = dict((k, frozenset(v.values)) for k, v in test_not_favourable.groupby("UserID")["MovieID"])
#test_users = test_dataset["UserID"].unique()

print len(test_favorable_by_users)

correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)

for user, reviews in test_favorable_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1
				
# Now, we compute the confidence of each of these rules. This is very similar to what we did in chapter 1
correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in favorable_reviews_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1
rule_confidence = {candidate_rule: correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
              for candidate_rule in candidate_rules}

			  
test_confidence = {candidate_rule: correct_counts[candidate_rule] / float(correct_counts[candidate_rule] + incorrect_counts[candidate_rule])
                   for candidate_rule in rule_confidence}
print(len(test_confidence))

sorted_test_confidence = sorted(test_confidence.items(), key=itemgetter(1), reverse=True)
print(sorted_test_confidence[:5])

for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    premise_names = ", ".join(get_movie_name(idx) for idx in premise)
    conclusion_name = get_movie_name(conclusion)
    print("Rule: If a person recommends {0} they will also recommend {1}".format(premise_names, conclusion_name))
    print(" - Train Confidence: {0:.3f}".format(rule_confidence.get((premise, conclusion), -1)))
    print(" - Test Confidence: {0:.3f}".format(test_confidence.get((premise, conclusion), -1)))
    print("")

62584
35
[((frozenset(['0345417623']), '0316666343'), 1.0), ((frozenset(['0446606812', '0316666343']), '0345417623'), 1.0), ((frozenset(['0446606812']), '0316666343'), 1.0), ((frozenset(['0446672211', '0345417623']), '0316666343'), 1.0), ((frozenset(['0971880107', '0446672211']), '0345417623'), 1.0)]
Rule #1
Rule: If a person recommends Timeline they will also recommend The Lovely Bones: A Novel
 - Train Confidence: 1.000
 - Test Confidence: 1.000

Rule #2
Rule: If a person recommends Message in a Bottle, The Lovely Bones: A Novel they will also recommend Timeline
 - Train Confidence: 1.000
 - Test Confidence: 1.000

Rule #3
Rule: If a person recommends Wild Animus, Timeline they will also recommend The Lovely Bones: A Novel
 - Train Confidence: 1.000
 - Test Confidence: 1.000

Rule #4
Rule: If a person recommends Message in a Bottle they will also recommend The Lovely Bones: A Novel
 - Train Confidence: 1.000
 - Test Confidence: 1.000

Rule #5
Rule: If a person recommends Where the He